<a href="https://colab.research.google.com/github/StellaIbeh/Data_preprocessing-Assignment-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [59]:
!kaggle datasets download -d thedevastator/comprehensive-medical-q-a-dataset



Dataset URL: https://www.kaggle.com/datasets/thedevastator/comprehensive-medical-q-a-dataset
License(s): CC0-1.0
comprehensive-medical-q-a-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [60]:
import zipfile
zip_ref = zipfile.ZipFile('/content/comprehensive-medical-q-a-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [36]:
!pip install transformers datasets evaluate nltk torch


In [61]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords') #Also download 'stopwords' since you use it

import re
import nltk
import torch
import pandas as pd
import evaluate
from datasets import Dataset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments
)
from transformers import DataCollatorForSeq2Seq
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Ensure stopwords are downloaded and initialized
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
# Load dataset
df = pd.read_csv("/content/train.csv", names=["Question", "Answer"])

# Randomly sample 4000 rows from the dataset
df_reduced = df.sample(n=4000, random_state=42)

print("Subset of 4k rows:")
print(df_reduced.head())


Subset of 4k rows:
                                                     Question  \
frequency    How many people are affected by Lujan syndrome ?   
symptoms             What are the symptoms of PEHO syndrome ?   
treatment     What are the treatments for Anxiety Disorders ?   
treatment   What are the treatments for generalized arteri...   
prevention  How to prevent Parasites - Enterobiasis (also ...   

                                                       Answer  
frequency   Lujan syndrome appears to be an uncommon condi...  
symptoms    What are the signs and symptoms of PEHO syndro...  
treatment   Most insurance plans, including health mainten...  
treatment   These resources address the diagnosis or manag...  
prevention  Washing your hands with soap and warm water af...  


In [67]:

df.head(4000)


,Question,Answer
qtype,Question,Answer
susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
...,...,...
susceptibility,Who is at risk for Skin Cancer? ?,The best way to reduce your skin cancer risk i...
information,What is (are) Skin Cancer ?,SPF is a sun protection rating. The SPF of a s...
symptoms,What are the symptoms of Skin Cancer ?,Skin cancer is usually visible. Changes in you...
information,What is (are) Skin Cancer ?,"Once cancer has been found, the doctor will ne..."


In [68]:
#Text cleaning
def clean_text(text):
    """Basic text cleaning: lowercasing, removing special characters, and extra spaces."""
    text = text.lower()
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = re.sub(r"[^a-zA-Z0-9?.!,]", " ", text)  # Keep only alphanumeric and punctuation
    text = text.strip()
    return text


In [69]:
#Text Normalization
def normalize_text(text):
    """Tokenization and stopword removal"""
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

In [70]:
# Apply cleaning and normalization
df["Question"] = df["Question"].apply(lambda x: normalize_text(clean_text(str(x))))
df["Answer"] = df["Answer"].apply(lambda x: normalize_text(clean_text(str(x))))

In [71]:
# Split Dataset into Train, Validation, and Test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["Question"].tolist(), df["Answer"].tolist(), test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

In [72]:
# Convert into Hugging Face Dataset format
dataset = DatasetDict({
    "train": Dataset.from_dict({"Question": train_texts, "Answer": train_labels}),
    "validation": Dataset.from_dict({"Question": val_texts, "Answer": val_labels}),
    "test": Dataset.from_dict({"Question": test_texts, "Answer": test_labels}),
})

In [73]:
# Load Model for QA Fine-tuning
model_name = "facebook/bart-base"  # Use a sequence-to-sequence model like BART
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [74]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [75]:
# Data collator (for padding dynamically during training)
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [76]:
def preprocess_function(examples):
    """Tokenization function for BioClinicalBERT"""
    inputs = ["question: " + q for q in examples["Question"]]
    targets = examples["Answer"]

    # Tokenizing question and answer
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [77]:
# Tokenize dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/13126 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

In [85]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    gradient_accumulation_steps=8,
    fp16=True,
    report_to="none"  # This disables Weights & Biases logging
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [86]:
# Define Evaluation Metrics (BLEU, F1, Perplexity)
bleu_metric = evaluate.load("bleu")
f1_metric = evaluate.load("f1")
perplexity_metric = evaluate.load("perplexity")


In [87]:
def compute_metrics(eval_pred):
    """Compute BLEU, F1 score, and Perplexity."""
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).tolist()
    labels = torch.tensor(labels).tolist()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    f1 = f1_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Compute Perplexity (Cross-Entropy Loss)
    loss_fct = CrossEntropyLoss()
    perplexity = torch.exp(loss_fct(torch.tensor(logits), torch.tensor(labels).view(-1))).item()

    return {
        "BLEU": bleu["bleu"],
        "F1 Score": f1["f1"],
        "Perplexity": perplexity
    }

In [88]:
# Train Model with Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


<ipython-input-88-95f82387389d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.13 GiB. GPU 0 has a total capacity of 14.74 GiB of which 5.12 GiB is free. Process 5565 has 9.62 GiB memory in use. Of the allocated memory 8.88 GiB is allocated by PyTorch, and 609.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Evaluate on Test Set
eval_results = trainer.evaluate(tokenized_datasets["test"])
print("Evaluation Results on Test Set:", eval_results)

In [ ]:
import torch
# ... load your model ...
torch.cuda.empty_cache()  # Free up cached memory
# ... start training ...

In [ ]:
# Save Model & Tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

In [ ]:
# Make Predictions
test_question = "What are the symptoms of heart disease?"
input_text = "question: " + test_question
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)

In [ ]:
# Generate answer
outputs = model.generate(input_ids, max_length=128, num_beams=2, early_stopping=True)
predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("User Query:", test_question)
print("Predicted Answer:", predicted_answer)